### Determining the optimal number of hidden layers and neurons for an Artificial Neural Network (ANN) 
This can be challenging and often requires experimentation. However, there are some guidelines and methods that can help you in making an informed decision:

- Start Simple: Begin with a simple architecture and gradually increase complexity if needed.
- Grid Search/Random Search: Use grid search or random search to try different architectures.
- Cross-Validation: Use cross-validation to evaluate the performance of different architectures.
- Heuristics and Rules of Thumb: Some heuristics and empirical rules can provide starting points, such as:
  -    The number of neurons in the hidden layer should be between the size of the input layer and the size of the output layer.
  -  A common practice is to start with 1-2 hidden layers.

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [5]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
#Preprocess the data
df = df.drop(['RowNumber','CustomerId','Surname'],axis=1)

#Encode categorical variable
label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])

#OneHot Encode Geographical Column
from sklearn.preprocessing import OneHotEncoder
onehot_encode_geo = OneHotEncoder()
geo_encoder = onehot_encode_geo.fit_transform(df[['Geography']])

geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encode_geo.get_feature_names_out(['Geography']))

#Combine  onehot encoder column with original data
df = pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)

#Divide the dataset into independent and dependent features
X = df.drop('Exited',axis=1)
y = df['Exited']

In [7]:
#Split the data in training and test set
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
#Save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encode_geo.pkl','wb') as file:
    pickle.dump(onehot_encode_geo,file) 

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)  

In [11]:
#Define a function to create a model and try different parameters
def create_model(neurons=32,layers=1):
    model = Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [16]:
#Create KerasClassifier
model = KerasClassifier(neurons=32,layers=1,build_fn=create_model,epochs=50,batch_size=10,verbose=0)

In [17]:
#Define the grid search parameters
param_grid = {
    'neurons': [12, 32, 64, 128],
    'layers': [1, 2, 3],
    'epochs': [50, 100]
}

In [ ]:
#Perform Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid,n_jobs=-1,cv=3)
grid_result = grid.fit(X_train,y_train)

#Print the best parametrs
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/Users/rishaan/miniforge3/envs/tensorflow-keras/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/rishaan/miniforge3/envs/tensorflow-keras/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
2024-12-12 02:51:50.113562: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-12-12 02:51:50.113588: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-12-12 02:51:50.113596: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-12-12 02:51:50.113626: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of pl